# UC15 Crop Conditions - sen2like processing

OpenEO provides the option to run the sen2like processor on Sentinel 2 and Landsat 8 & 9 data to produce L2F data for the requested spatio-temporal extent. The backend takes care of getting the required input data and the computational resources to run sen2like on. 

Based on the resulting L2F product, multiple indices can be computed, such as the NDVI and LAI. This is done in a separate follow-up notebook (UC15-indices.ipynb).

To start the sen2like openEO processing, we first connect to the openEO backend.

In [1]:
import openeo
from openeo.rest.datacube import THIS

conn = openeo.connect("openeo-dev.eodc.eu/openeo/1.1.0/").authenticate_oidc()

Authenticated using refresh token.


We select the "SENTINEL2_L1C" collection, specify the spatial and temporal extent and the bands to compute. The sen2like processing automatically includes the Landsat 8 & 9 data into the computation, so we do not need to call it explicitely. The processing also automatically includes other additional data, such as the digital elevation model and data from the Copernicus Atmosphere Monitoring Service.

In [2]:
spatial_extent = {"west": 16.3, "east": 16.5, "south": 48.6, "north": 48.7}
temporal_extent = ["2023-06-01", "2023-09-30"]

collection      = 'SENTINEL2_L1C'
bands = ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"]

S2 = conn.load_collection(
    collection, 
    spatial_extent=spatial_extent, 
    temporal_extent=temporal_extent, 
    bands=bands)

When applying the sen2like processing, we specify the "L2F" product and we set the "export_original_files" parameter to True. 

The cloud_cover can be between 1 and 100. The lower the value, the lower the cloud cover for the files - files with higher cloud cover are ignored.

In [3]:
sen2like = S2.process('sen2like', {
    'data': THIS,
    'target_product': 'L2F', 
    'export_original_files': True,
    'cloud_cover': 50})

We create and start the openEO job.

In [4]:
job = sen2like.create_job().start_job()

Preflight process graph validation raised: [404] Load call not available for SENTINEL2_L1C


In [5]:
job

<BatchJob job_id='685d7e0e-c652-4cec-a33e-28af61cbcfb8'>

Once the job status is "finished", we can download and explore the results. 

We reuse the results in the "UC15-indices.ipynb", so we need to know the job id from this notebook. For this run it is "685d7e0e-c652-4cec-a33e-28af61cbcfb8".

In [ ]:
results = job.get_results().download_files("sen2like_outputs")

#### Explore the openEO results - Create RGB imagery

In [ ]:
import numpy as np 
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
data = xr.open_mfdataset("EU010M_E051N014T1_20230930T000000.nc")

data

In [ ]:
brg = np.zeros((data.B04.shape[0],data.B04.shape[1],3))
brg[:,:,0] = (data.B04.values+1)/15
brg[:,:,1] = (data.B03.values+1)/14
brg[:,:,2] = (data.B02.values+1)/11.6
brg = np.clip(brg,0,255).astype(np.uint8)

plt.figure(figsize=(8,12))
plt.title("RGB sen2like, Austria 2023")
plt.imshow(brg,cmap='brg')

#### Running UDPs in the webeditor

There is an option, to run the sen2like processor in the openEO webeditor as a UDP: https://editor.openeo.org/?server=https%3A%2F%2Fopeneo.eodc.eu%2Fopeneo%2F1.1.0&wizard=UDP&wizard%7Eprocess=sen2like_original_outputs 

In the openEO wizard, the UDP sen2like_original_outputs can be seleted, where we only need to specify the "spatial_extent", "temporal_extent" and "bands".

<img src="../udp-1.png" width="1200"/>